In [ ]:
import os
import matplotlib.pyplot as plt
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy
from QAOA_simulator import get_simulator, get_expectation, get_result
from plot_utils import plot_heat_map


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> np.ndarray:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np.ndarray:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np.ndarray:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


# Define parameter ranges
gammas = np.linspace(0, np.pi, 50)
betas = np.linspace(0, np.pi, 50)

# Probabilities for the Erdos-Renyi graph generation
# probabilities = [0.1, 0.25, 0.5, 0.75, 0.9]

for p in [0.5]:
    for N in range(2, 31):
        G = nx.erdos_renyi_graph(N, p, seed=18) # generate graphs
        M = G.number_of_edges()

        # Define paths
        base_path = f"data_for_Expectation_Heatmaps/Erdős_Rényi/ER_p={p}"
        qaoa_path = os.path.join(base_path, f"QAOA_N={N}_M={M}.npz")
        paper_proxy_path = os.path.join(base_path, f"Paper_Proxy_N={N}_M={M}.npz")
        our_proxy_path = os.path.join(base_path, f"Our_Proxy_N={N}_M={M}.npz")

        # Create directories if they do not exist
        os.makedirs(base_path, exist_ok=True)

        # if os.path.exists(qaoa_path):
        #     data = np.load(qaoa_path)
        #     expectations = data['expectations']
        #     gammas = data['gammas']
        #     betas = data['betas']
        # else:
        #     expectations = collect_parameter_data(G, gammas, betas)
        #     np.savez(qaoa_path, expectations=expectations, gammas=gammas, betas=betas)

        if os.path.exists(paper_proxy_path):
            data = np.load(paper_proxy_path)
            expectation_proxies = data['expectations']
            gammas = data['gammas']
            betas = data['betas']
        else:
            expectation_proxies = collect_parameter_data_proxy(M, N, gammas, betas)
            np.savez(paper_proxy_path, expectations=expectation_proxies, gammas=gammas, betas=betas)

        if os.path.exists(our_proxy_path):
            data = np.load(our_proxy_path)
            expectation_proxies_less_dumb = data['expectations']
            gammas = data['gammas']
            betas = data['betas']
        else:
            expectation_proxies_less_dumb = collect_parameter_data_proxy_less_dumb(M, N, gammas, betas)
            np.savez(our_proxy_path, expectations=expectation_proxies_less_dumb, gammas=gammas, betas=betas)

         # Define image save paths
        img_base_path = f"Expectation_Heatmaps/Erdős_Rényi/ER_p={p}"
        qaoa_img_path = os.path.join(img_base_path, f"QAOA_N={N}_M={M}.png")
        paper_proxy_img_path = os.path.join(img_base_path, f"paper_proxy_N={N}_M={M}.png")
        our_proxy_img_path = os.path.join(img_base_path, f"our_proxy_N={N}_M={M}.png")

        # Create directories for images if they do not exist
        os.makedirs(img_base_path, exist_ok=True)

        # Generate and save heatmaps
        # _ = plot_heat_map(gammas, betas, expectations, f"True QAOA Expectation (N={N},M={M})", "Gamma", "Beta")
        # plt.savefig(qaoa_img_path)
        _ = plot_heat_map(gammas, betas, expectation_proxies, f"Expectation Proxies from Paper (N={N},M={M})", "Gamma", "Beta")
        plt.savefig(paper_proxy_img_path)
        _ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, f"Expectation Proxies from us (N={N},M={M})", "Gamma", "Beta")
        plt.savefig(our_proxy_img_path)

        plt.show()